# IMPORTS

In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os

In [2]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
from metrics import compute_single_col_score, get_tresholds
from helper import make_prediction
from sklearn.model_selection import StratifiedKFold, train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [3]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu

# MODEL TRAINING

In [4]:
train = pd.read_pickle(cfg.PREPARED_TRAIN_DATA_PATH)
test = pd.read_pickle(cfg.PREPARED_TEST_DATA_PATH)

In [5]:
X_train, Y_train = train.drop(cfg.TARGETS, axis=1), train[cfg.TARGETS]

In [6]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(cfg.TARGETS))), index=train.index, columns=cfg.TARGETS)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(cfg.TARGETS))), index=test.index, columns=cfg.TARGETS)
metrics = {}

In [7]:
EXPERIMENT_FAMILY_NAME = 'catboost'
EXPERIMENT_NAME = 'baseline'
RANDOM_STATE = 77
N_SPLITS = 5
N_RANDOM_SEEDS = 7

In [8]:
cv = MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

CAT_COLS = cfg.CAT_UNORDERED_COLS

test_pool = cb.Pool(
        data=test,
        cat_features=CAT_COLS)


fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, Y_train), total=N_SPLITS):

    train_pool = cb.Pool(
        data=X_train.iloc[train_idx], 
        label=Y_train.iloc[train_idx],
        cat_features=CAT_COLS)

    val_pool = cb.Pool(
        data=X_train.iloc[val_idx], 
        label=Y_train.iloc[val_idx],
        cat_features=CAT_COLS)
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = cb.CatBoostClassifier(
            loss_function='MultiLogloss',
            custom_metric=['Recall', 'F1'],
            iterations=2000,
            silent=True,
            depth=6,
            l2_leaf_reg=2.0,
            learning_rate=0.001,
            early_stopping_rounds=100,
            random_seed=random_seed
        )

        clf.fit(train_pool, eval_set=val_pool, plot=False)
        
        model_path = os.path.join(
            cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME, 
            f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}.cbm')
        clf.save_model(model_path)
        
        pred_proba_oof.iloc[val_idx, :] += clf.predict_proba(val_pool)
        pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (N_SPLITS * N_RANDOM_SEEDS)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

# PREDICT AND SAVE PREDICTIONS

In [ ]:
tresholds = get_tresholds(train[cfg.TARGETS], pred_proba_oof)
sample_submission = pd.read_csv(cfg.SAMPLE_SUBMISSION_PATH).set_index('ID')
submission = make_prediction(pred_proba_test, tresholds, sample_submission)

0.6880135426726085


In [ ]:
## BEST PARAMS
# 0.6880135426726085

# RANDOM_STATE = 77
# N_SPLITS = 7

# iterations=1000,
# silent=True,
# depth=6,
# l2_leaf_reg=2.0,
# learning_rate=0.01,
# early_stopping_rounds=100

In [ ]:
submission.to_csv(os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME, f'{EXPERIMENT_NAME}.csv'))
pred_proba_oof.to_pickle(os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME, f'{EXPERIMENT_NAME}.pkl'))
pred_proba_test.to_pickle(os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME, f'{EXPERIMENT_NAME}.pkl'))